### Imports & loading data

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_parquet('yellow_tripdata_2023-01.parquet', engine='pyarrow')
df_val = pd.read_parquet('yellow_tripdata_2023-02.parquet', engine='pyarrow')

### Q1: Get number of columns for January dataset => 19

In [3]:
df.shape

(3066766, 19)

### Q2: Add duration of a ride as column. What's the stdev of ride duration in January? => 42.59

In [4]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime

df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)



In [12]:
count_full = (df.duration.describe().round(2))[0]
stdev_full = (df.duration.describe().round(2))[2]
count_full, stdev_full

C:\Users\SebastianSy\AppData\Local\Temp\ipykernel_11580\2168452889.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  count_full = (df.duration.describe().round(2))[0]
C:\Users\SebastianSy\AppData\Local\Temp\ipykernel_11580\2168452889.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  stdev_full = (df.duration.describe().round(2))[2]


(3066766.0, 42.59)

### Q3:Drop outliers (rides shorter than 1min and longer than 60min). What fraction of rides remains?

In [36]:
df = df[(df.duration >= 1) & (df.duration <= 60)]
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]
count_outliers_removed = (df.duration.describe().round(2))[0]


C:\Users\SebastianSy\AppData\Local\Temp\ipykernel_11580\1205340321.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  count_outliers_removed = (df.duration.describe().round(2))[0]


In [14]:
count_outliers_removed/count_full*100

98.1220282212598

In [18]:
df_val.shape

(2913955, 20)

### Q4: One-hot encoding of pickup and dropofflocation IDs. How many cols does the feature matrix have? =>515

In [37]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)

categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)
df_val[categorical] = df_val[categorical].astype(str)

train_dicts = df[categorical].to_dict(orient='records')
val_dicts = df_val[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)

target = 'duration'
y_train = df[target].values
y_val = df_val[target].values


C:\Users\SebastianSy\AppData\Local\Temp\ipykernel_11580\154925134.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val[categorical] = df_val[categorical].astype(str)


In [38]:
X_train, X_val

(<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
 	with 6018346 stored elements in Compressed Sparse Row format>,
 <2855951x515 sparse matrix of type '<class 'numpy.float64'>'
 	with 5711894 stored elements in Compressed Sparse Row format>)

### Q5: Train a model using the feature matrix and linearRegression. What's the RMSE on the training data?

In [40]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)



7.649261952121727

### Q6: Evaluate the model on the validation dataset (Feb). What's the RMSE?

In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)


7.811816484222663